In [1]:
import os, math
import _pickle as pickle
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
dir_path = '/home/chenyitian/TCNPaperExperiments/EC/data'
dt = np.loadtxt(os.path.join(dir_path,"modelData.csv"), dtype=np.str, delimiter=",")

In [3]:
## The date range
date_list = pd.date_range(start='01/01/2012', end='31/12/2014')
date_list = pd.to_datetime(date_list)

hour_list = []
for nDate in date_list:
    for nHour in range(24):
        tmp_timestamp = nDate+timedelta(hours=nHour)
        hour_list.append(tmp_timestamp)
hour_list = np.array(hour_list)

In [4]:
station_index = list(range(370))

sliding_window_dis = 24;
sample_len = 192; #168+24
input_len = 168;
output_len = 24;
total_n = 800; ## The total days
test_n = 7     ## The testing days, day of the last 7 days
train_n = total_n - test_n ## The training days

In [5]:
trainX_list = [];trainX2_list = [];trainY_list = [];trainY2_list = []
testX_list = [];testX2_list = [];testY_list = [];testY2_list = []
#for station in station_index:
for station in station_index:
    print(station)
    sub_series = dt[station,1:].astype('float32')
    sub_index = np.array(range(26304))-np.min(np.where(sub_series>0))
    trainX = np.zeros(shape=(train_n, input_len))       ## The input series
    trainY = np.zeros(shape=(train_n, output_len))      ## The output series  

    testX  = np.zeros(shape=(7, input_len))        ## The input series
    testY = np.zeros(shape=(7, output_len))        ## The output series

    covariate_num = 8   # other features covariate_num: sub_index, station_id,nYear,nMonth,day_of_month, day_of_week, iHour
    trainX2 = np.zeros(shape=(train_n, input_len,covariate_num))
    trainY2 = np.zeros(shape=(train_n, output_len,covariate_num))
    testX2 = np.zeros(shape=(7, input_len,covariate_num))
    testY2 = np.zeros(shape=(7, output_len,covariate_num))
    ### Testing samples (7+1)*24
    ts_len = sub_series.shape[0]
    start_index = ts_len-sample_len
    for i in range(total_n):
        ### The sequence data
        series_x = sub_series[start_index:start_index+input_len]
        series_y = sub_series[start_index+input_len:start_index+sample_len]
        ### The index data
        hour_mean = np.mean(series_x.reshape(-1,24),axis=0)
        index_x = np.tile(hour_mean,7)
        index_y = np.tile(hour_mean,1)
        ### The covariate
        station_X = np.repeat(station, input_len)
        station_Y = np.repeat(station, output_len)
        ### the time index
        time_index_x = pd.to_datetime(hour_list[start_index:start_index+input_len])
        time_index_y = pd.to_datetime(hour_list[start_index+input_len:start_index+sample_len])
        nYear_X, nYear_Y = time_index_x.year-2012, time_index_y.year-2012
        nMonth_X, nMonth_Y = time_index_x.month-1, time_index_y.month-1
        mDay_X, mDay_Y = time_index_x.day-1, time_index_y.day-1
        wDay_X, wDay_Y = time_index_x.weekday, time_index_y.weekday
        nHour_X, nHour_Y = time_index_x.hour, time_index_y.hour
        holiday_X,holiday_Y = (mDay_X==24),(mDay_Y==24)
    
        covariate_X = np.c_[station_X,index_x,nYear_X,nMonth_X,mDay_X,wDay_X,nHour_X,holiday_X]
        covariate_Y = np.c_[station_Y,index_y,nYear_Y,nMonth_Y,mDay_Y,wDay_Y,nHour_Y,holiday_Y]
    
        if(i<test_n):
            test_index = i
            testX[test_index] = series_x
            testY[test_index] = series_y
            testX2[test_index] = covariate_X
            testY2[test_index] = covariate_Y
        
        else:
            trainX[i-test_n] = series_x
            trainY[i-test_n] = series_y
            trainX2[i-test_n] = covariate_X
            trainY2[i-test_n] = covariate_Y
        # update the start_index
        start_index = start_index - sliding_window_dis
  

    testX_list.append(testX)
    testX2_list.append(testX2)
    testY_list.append(testY)
    testY2_list.append(testY2)
    
    trainX_list.append(trainX)
    trainX2_list.append(trainX2)
    trainY_list.append(trainY)
    trainY2_list.append(trainY2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
testX_dt = np.vstack(testX_list)
testY_dt = np.vstack(testY_list)
testX2_dt = np.vstack(testX2_list)
testY2_dt = np.vstack(testY2_list)

trainX_dt = np.vstack(trainX_list)
trainY_dt = np.vstack(trainY_list)
trainX2_dt = np.vstack(trainX2_list)
trainY2_dt = np.vstack(trainY2_list)

In [7]:
testY_dt.shape
### Save the data
#with open('tensor_feature.pkl', 'wb') as f:
#    pickle.dump([trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt], f, -1)
#with open('tensor_feature.pkl', 'rb') as f:
#    [trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt] = pickle.load(f)

(2590, 24)

In [8]:
scaler = preprocessing.StandardScaler()
scaler.fit(trainX_dt)

trainX_dt = scaler.transform(trainX_dt)
testX_dt = scaler.transform(testX_dt)
scaler2 = preprocessing.StandardScaler()
scaler2.fit(trainX2[:,:,1])
trainX2[:,:,1] = scaler2.transform(trainX2[:,:,1])
testX2[:,:,1] = scaler2.transform(testX2[:,:,1])

### The filter data
### Select the data of the Nov
isNov = trainX2_dt[:,:,3]>=11
trainX_dt = trainX_dt[isNov[:,0]]
trainY_dt = trainY_dt[isNov[:,0]]
trainX2_dt = trainX2_dt[isNov[:,0]]
trainY2_dt = trainY2_dt[isNov[:,0]]
### Save the data
with open('feature_prepare.pkl', 'wb') as f:
    pickle.dump([trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt], f, -1)